<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

**Author: Tomasz Pawłowski**



# LSML - LAB 1

Plan

1. Setting up pyspark
2. Map-Reduce word count example
3. Map-Reduce exercises
4. Homework



To **edit the colab** first copy it to your drive with the top bars `File -> Save a copy on drive` option.

## Setting up PySpark


In [ ]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet
!apt install openjdk-8-jdk-headless &> /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').setAppName("mlibs").setMaster("local[2]")
sc = SparkContext.getOrCreate(conf=conf)

Note: pyspark documentation is [here](https://spark.apache.org/docs/3.1.2/api/python/reference/index.html). For example: list of `SparkContext` methods is [here](https://spark.apache.org/docs/3.1.2/api/python/reference/pyspark.html#spark-context-apis).


## Example: Loading data

*Loading* data to clusters RAM. In spark such distributed datasets are called [RDD](https://spark.apache.org/docs/latest/rdd-programming-guide.html#resilient-distributed-datasets-rdds). They are divided into partitions which can be processed independently by different workers.

`RDD` can be created from python collection, or by loading data from a file.


### From python iterable

In [ ]:
number_of_partitions = 4

array_of_numbers = [1, 2, 3, 4, 5]
rdd_of_numbers = sc.parallelize(array_of_numbers, number_of_partitions)

# `glom` shows how data is distributed in partitions
rdd_of_numbers.glom().collect()

[[1], [2], [3], [4, 5]]

### From google drive

Loading data from google drive. Based on [tutorial](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=eFOvsAYk1tcH). Note that loading from google drive is useful but not required for this lab. You can also upload files to colab using the left menu bars `Files` menu, but such files are not persisted.

Note that using google drive from other account that from which you open notebooks may not work.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Your google drive should now be mounted in the `/content/drive/My Drive/` local directory of this colab. The next code fragment assumes that you have there the following directory/file: `dir_on_my_drive/input.txt`.

In [ ]:
import os

# first upload input.txt to your gdrive!
path = "/content/drive/My Drive/dir_on_my_drive/input.txt"
if os.path.isfile(path):
  lines_from_drive_file = sc.textFile(path)
  print(lines_from_drive_file.collect())
  # ...

In [ ]:
drive.flush_and_unmount()

### From online resource

Downloading data to colab (machine executing the colab's) from an online source can be done using `wget` shell tool. Note the exclamation mark (!) before the command which indicates that this line should be executed by shell not python.

Note that such downloaded files are not persisted. They are deleted after closing the colab session.

In [ ]:
!wget -q https://wolnelektury.pl/media/book/txt/balladyna.txt -O sample_data/balladyna.txt
!wget -q https://wolnelektury.pl/media/book/txt/zemsta.txt -O sample_data/zemsta.txt

After downloading such files are visible in left menu bars `Files` section in `sample_data` directory.

They can be accessed from python code using path: `sample_data/balladyna.txt`.

In [ ]:
with open('sample_data/balladyna.txt', 'r') as f:
  for i, line in enumerate(f.readlines()):
    print(f"{i}: {line}", end='')
    if i > 15:
      break

0: Juliusz Słowacki
1: 
2: Balladyna
3: Tragedia w pięciu aktach
4: 
5: ISBN 978-83-288-2852-0
6: 
7: 
8: 
9: 
10: KOCHANY POETO RUIN!
11: 
12: Pozwól, że pisząc do ciebie, zacznę od apologu, który mi opowiedziano nad Salaminy zatoką.
13: 
14: Stary i ślepy harfiarz z wyspy Scio przyszedł nad brzegi Morza Egejskiego, a usłyszawszy z wielkim hukiem łamiące się fale, myślał, że szum ów pochodził od zgiełku ludzi, którzy się zbiegli pieśni rycerskich posłuchać. — Oparł się więc na harfie i śpiewał pustemu morza brzegowi: a kiedy skończył, zadziwił się, że żadnego ludzkiego głosu, żadnego westchnienia, żadnego pieśń nie zyskała oklasku. Rzucił więc harfę precz daleko od siebie, a te fale, które śpiewak mniemał tłumem ludzkim, odniosły złote pieśni narzędzie i położyły mu je przy stopach. I odszedł od harfy swojej smutny Greczyn, nie wiedząc, że najpiękniejszy rapsod nie w sercach ludzi, ale w głębi fal Egejskiego Morza utonął.
15: 
16: Kochany Irydionie! Ta powiastka o falach i harfiarzu z

## Example: Saving data

There are multiple save methods in [RDD API](https://spark.apache.org/docs/3.1.2/api/python/reference/pyspark.html#rdd-apis)

Here are some examples. After executing following two code examples check how the data is formatted.

In [ ]:
# Saving as text file

sc.parallelize([('a', 1), ('b', 2), ('c', 0), ('d', 1), ('e', 0)], 2).saveAsTextFile('save_example1')

In [ ]:
# Saving as pickle

sc.parallelize([('x', 1), ('y', 2), ('z', 0), ('w', 1), ('ź', 4)], 2).saveAsPickleFile('save_example2')

Note that partitions are saved separately in their own directoires.

Loading of pickled files is straightforward. For text files it can be loaded with [textFile](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.SparkContext.textFile.html#pyspark.SparkContext.textFile) but requires manual parsing. `glom` method used below visualizes how data is split between partitions.

In [ ]:
print(sc.pickleFile('save_example2').glom().collect())

[[('z', 0), ('w', 1), ('ź', 4)], [('x', 1), ('y', 2)]]


## Map-Reduce sample interface

You don't need to understand it, yet. Just execute this cell. The `map_reduce` method will be used in the following examples and exercises.

In [ ]:
import itertools
import operator
from pyspark import RDD
from typing import Callable, Iterable, List, Tuple, TypeVar

T = TypeVar('T')
U = TypeVar('U')
V = TypeVar('V')
S = TypeVar('S')

def map_reduce(
    input: RDD[T],
    map_function: Callable[[T], Iterable[Tuple[U, V]]],
    reduce_function: Callable[[U, Iterable[V]], Iterable[S]],
    combiner_function: Callable[[U, Iterable[V]], V] = None
    ) -> RDD[S]:

  mapped_rdd = input.flatMap(map_function)

  if combiner_function is not None:
    def apply_combiner(partition: Iterable[Tuple[U, V]]) -> Iterable[Tuple[U, V]]:
      it = itertools.groupby(partition, operator.itemgetter(0))
      for key, values in it:
        yield key, combiner_function(key, [v for _, v in values])

    mapped_rdd = mapped_rdd.mapPartitions(apply_combiner)

  return mapped_rdd\
      .groupByKey()\
      .flatMap(lambda k_v: reduce_function(k_v[0], k_v[1]))


## Example: Word count

In [ ]:
lines_from_file = sc.textFile('sample_data/balladyna.txt')

lines_from_file.take(20)

['Juliusz Słowacki',
 '',
 'Balladyna',
 'Tragedia w pięciu aktach',
 '',
 'ISBN 978-83-288-2852-0',
 '',
 '',
 '',
 '',
 'KOCHANY POETO RUIN!',
 '',
 'Pozwól, że pisząc do ciebie, zacznę od apologu, który mi opowiedziano nad Salaminy zatoką.',
 '',
 'Stary i ślepy harfiarz z wyspy Scio przyszedł nad brzegi Morza Egejskiego, a usłyszawszy z wielkim hukiem łamiące się fale, myślał, że szum ów pochodził od zgiełku ludzi, którzy się zbiegli pieśni rycerskich posłuchać. — Oparł się więc na harfie i śpiewał pustemu morza brzegowi: a kiedy skończył, zadziwił się, że żadnego ludzkiego głosu, żadnego westchnienia, żadnego pieśń nie zyskała oklasku. Rzucił więc harfę precz daleko od siebie, a te fale, które śpiewak mniemał tłumem ludzkim, odniosły złote pieśni narzędzie i położyły mu je przy stopach. I odszedł od harfy swojej smutny Greczyn, nie wiedząc, że najpiękniejszy rapsod nie w sercach ludzi, ale w głębi fal Egejskiego Morza utonął.',
 '',
 'Kochany Irydionie! Ta powiastka o falach i har

In [ ]:
import re

example_words = lines_from_file.flatMap(lambda line: filter(None, re.split(r'\s+', line.lower())))

example_words.take(30)

['juliusz',
 'słowacki',
 'balladyna',
 'tragedia',
 'w',
 'pięciu',
 'aktach',
 'isbn',
 '978-83-288-2852-0',
 'kochany',
 'poeto',
 'ruin!',
 'pozwól,',
 'że',
 'pisząc',
 'do',
 'ciebie,',
 'zacznę',
 'od',
 'apologu,',
 'który',
 'mi',
 'opowiedziano',
 'nad',
 'salaminy',
 'zatoką.',
 'stary',
 'i',
 'ślepy',
 'harfiarz']

In [ ]:
from pyspark import RDD
from typing import Tuple

def word_count(words_rdd: RDD[str]) -> RDD[Tuple[str, int]]:
  return map_reduce(
      words_rdd,
      lambda word: [(word, 1)],
      lambda k, vs: [(k, sum(vs))]
  )

word_count(example_words).take(15)

[('juliusz', 3),
 ('balladyna', 263),
 ('kochany', 2),
 ('ruin!', 1),
 ('pozwól,', 2),
 ('do', 229),
 ('ciebie,', 7),
 ('zacznę', 1),
 ('apologu,', 1),
 ('mi', 89),
 ('salaminy', 1),
 ('i', 518),
 ('ślepy', 1),
 ('harfiarz', 1),
 ('scio', 1)]

In [ ]:
def word_count_with_combiner(words_rdd: RDD[str]) -> RDD[Tuple[str, int]]:
  return map_reduce(
      words_rdd,
      lambda word: [(word, 1)],
      lambda k, vs: [(k, sum(vs))],
      lambda _, vs: sum(vs)
  )

word_count_with_combiner(example_words).take(15)

[('juliusz', 3),
 ('balladyna', 263),
 ('kochany', 2),
 ('ruin!', 1),
 ('pozwól,', 2),
 ('do', 229),
 ('ciebie,', 7),
 ('zacznę', 1),
 ('apologu,', 1),
 ('mi', 89),
 ('salaminy', 1),
 ('i', 518),
 ('ślepy', 1),
 ('harfiarz', 1),
 ('scio', 1)]

## Exercise 1: Average

Calculate average for given `RDD` of numbers. Use `map_reduce` utility method. Try to add a combiner function.

In [ ]:
from typing import Union

def average(numbers_rdd: RDD[Union[int, float]]) -> float:
  # TODO: write your code here
  pass

In [ ]:
[
    average(sc.parallelize([1,2,3,4,5,6])), # should be 3.5
    average(sc.parallelize([42, 12])),      # should be 27 or 27.0
    average(sc.parallelize([]))             # NaN or raise a meaningful exception
]

[3.5, 27.0, nan]

## Exercise 2: Jaccard similarity coefficient

Compute [Jaccard similarity coefficient](https://en.wikipedia.org/wiki/Jaccard_index) of two sets given by two RDD-s. Note: [union](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.SparkContext.union.html#pyspark.SparkContext.union) function may be needed. Use either `map_reduce` utility function or `map`, `flatMap`, `groupByKey` or other methods from [RDD](https://spark.apache.org/docs/3.1.2/api/python/reference/pyspark.html#rdd-apis) directly.

You can assume that both given `RDD`-s do not contain any duplicates.

In [ ]:
from pyspark import RDD
from typing import TypeVar

T = TypeVar('T')

def jaccard_similarity(a: RDD[T], b: RDD[T]) -> float:
  # TODO: write your code here
  pass

In [ ]:
[
    jaccard_similarity(sc.parallelize([1,2,3]), sc.parallelize([2,3,4])), # 0.5
    jaccard_similarity(sc.parallelize([1,2,3]), sc.parallelize([4,5])), # 0.0
    jaccard_similarity(sc.parallelize(['a', 'b', 'c', 'd']), sc.parallelize(['c', 'b'])), # 0.5
    jaccard_similarity(sc.parallelize([]), sc.parallelize([])) # NaN or raise a meaningful exception
]

[0.5, 0.0, 0.5, nan]

## Exercise 3: Fetching online data

Distribute fetching definitions of words from an online resource [Słownik języka polskiego](https://sjp.pl).

Cell below downloads a database of words tagged as acceptable in word games.

In [ ]:
!wget -q https://sjp.pl/sl/growy/sjp-20230709.zip -O sample_data/sjp-20230709.zip
!rm -f sample_data/slowa.txt
!unzip sample_data/sjp-20230709.zip slowa.txt -d sample_data

Archive:  sample_data/sjp-20230709.zip
  inflating: sample_data/slowa.txt   


In [ ]:
# prints example values

words_rdd = sc.textFile('sample_data/slowa.txt')
print(words_rdd.take(15))

words_sample_rdd = words_rdd.sample(False, 20.0/words_rdd.count())
print(words_sample_rdd.collect())

['aa', 'aaa', 'aalborscy', 'aalborska', 'aalborską', 'aalborski', 'aalborskich', 'aalborskie', 'aalborskiego', 'aalborskiej', 'aalborskiemu', 'aalborskim', 'aalborskimi', 'aalborsko', 'aalborsku']
['adorowaliby', 'betainom', 'czechizowanej', 'formalizmom', 'interplanetarni', 'miczmańskich', 'nacechowałbym', 'niejebiąca', 'niesfingowanych', 'odejmować', 'oleśniańskiej', 'ortografią', 'poleskiej', 'powiadomiona', 'pozwiększajcież', 'przekiwano', 'przyrostu', 'ranwers', 'rozwichrzane', 'rybiarzowi', 'sesyjnych', 'szklarnikach', 'wrzutowych', 'wskrzesiliście']


Next cell shows how to download a definitions of the word from the same source by parsing a html webpage.

In [ ]:
from pyspark import RDD
import re
import ssl
import urllib.parse
import urllib.request


def fetch_definition(word: str) -> Iterable[str]:
  definition_re = re.compile(r"<p style=\"margin: \.5em 0; font: medium[^\"]*\">([^<]+(?:<br />[^<]+)*)</p>")
  url = f"https://sjp.pl/{urllib.parse.quote(word)}"
  response = urllib.request.urlopen(url, context=ssl._create_unverified_context())
  html = response.read().decode(response.headers.get_content_charset())
  return (d.group(1).replace('<br />', "\n") for d in definition_re.finditer(html))

print(list(fetch_definition("test")))
print(list(fetch_definition("a kuku")))

['1. rodzaj pisemnego sprawdzianu, egzamin; zestaw odpowiednio przygotowanych pytań;\n2. badanie, próba;\n3. kontrolny obraz w odbiorniku telewizyjnym', 'partia wokalna w oratoriach, pełniąca funkcję narratora objaśniającego tło akcji i zaistniałą sytuację dramatyczną']
['wyrażenie mające na celu zasygnalizowanie swojej obecności osobie niespodziewającej się tego']


Using our `map_reduce` method and the function above a method that distributes downloading of the definitions of words can be created. Implement it as an exercise.

In [ ]:
def fetch_definitions(rdd: RDD[str]) -> RDD[Tuple[str, str]]:
  # TODO: write your code here
  pass

In [ ]:
word_definitions = fetch_definitions(words_sample_rdd)

for word, definition in word_definitions.collect():
  print(word)
  print(" ", definition.replace("\n", "\n  "))
  print("")

poleskiej
  przymiotnik od: Polesie

powiadomiona
  dać komuś o czymś znać, przekazać wiadomości

miczmańskich
  dotyczący miczmana

ortografią
  1. zbiór zasad i przepisów obowiązujących w danym języku, dotyczących poprawnej pisowni;
  2. sposób pisania wyrazów; grafia, pisownia

ranwers
  dawniej: wywinięty wyłóg w żakiecie lub w bluzce damskiej

niesfingowanych
  świadomie sfałszować lub upozorować coś

czechizowanej
  poddawać wpływom czeskim, rozszerzać wpływy czeskie; czeszczyć

formalizmom
  1. przywiązywanie nadmiernej wagi do ustalonych form i przepisów; 
  2. stanowisko w filozofii, sztuce, według którego wartość dzieła wynika w zasadniczym stopniu z jego formy;
  3. w nauce: matematyczne sformułowanie zagadnienia

interplanetarni
  międzyplanetarny

betainom
  związek organiczny, metylowa pochodna aminokwasu, występująca w małych ilościach w tkankach zwierzęcych, w znacznie większych - w roślinnych, mająca zastosowanie w medycynie i metalurgii

nacechowałbym
  1. nadać okreś

### Additional exercise:

Print words from `words_sample_rdd` for which no definition was found.

In [ ]:
from typing import Tuple

def words_without_any_definition(words: RDD[str], word_definitions: RDD[Tuple[str, str]]) -> RDD[str]:
  # TODO: write your code here
  pass

In [ ]:
print(f"words without any definition: {words_without_any_definition(words_sample_rdd, word_definitions).collect()}")

words without any definition: ['wrzutowych', 'sesyjnych', 'pozwiększajcież', 'odejmować', 'rozwichrzane', 'przyrostu', 'przekiwano']


## Homework 1: Set union, intersection and difference

Given two sets in `RDD` format calculate their union, intersection and difference.

Note: you can assume that there are no duplicates in given `RDD`-s. It is part of the exercise to make sure there will be no duplicates in the results.

In [ ]:
from pyspark import RDD
from typing import TypeVar


T = TypeVar('T')

# calculates: a ∪ b
def union(a: RDD[T], b: RDD[T]) -> RDD[T]:
  # TODO: write your code here
  pass

# calculates: a ∩ b
def intersection(a: RDD[T], b: RDD[T]) -> RDD[T]:
  # TODO: write your code here
  pass

# calculates: a - b
def difference(a: RDD[T], b: RDD[T]) -> RDD[T]:
  # TODO: write your code here
  pass

In [ ]:
# Write more tests to make sure your solution works great

a = sc.parallelize([1, 2, 3, 4])
b = sc.parallelize([3, 4, 5])

[
    union(a, b).collect(),
    intersection(a, b).collect(),
    difference(a, b).collect(),
    union(a, sc.emptyRDD()).collect(),
    union(sc.emptyRDD(), a).collect(),
    union(sc.emptyRDD(), sc.emptyRDD()).collect(),
    intersection(a, sc.emptyRDD()).collect(),
    intersection(sc.emptyRDD(), a).collect(),
    intersection(sc.emptyRDD(), sc.emptyRDD()).collect(),
    difference(a, sc.emptyRDD()).collect(),
    difference(sc.emptyRDD(), a).collect(),
    difference(sc.emptyRDD(), sc.emptyRDD()).collect()
]

## Homework 2: Join

Write a function calculating natural join of two relations.

In [ ]:
from pyspark import RDD
from typing import Tuple, TypeVar

T = TypeVar('T')
U = TypeVar('U')
V = TypeVar('V')

# calculates all triples (x, y, z) such that (x,y)∈r and (y,z)∈s
def join(r: RDD[Tuple[T, U]], s: RDD[Tuple[U, V]]) -> RDD[Tuple[T, U, V]]:
  # TODO: write your code here
  pass

In [ ]:
# Write more tests to make sure your solution works great

a = sc.parallelize([(1, 'a'), (0, 'b'), (1, 'b')])
b = sc.parallelize([('a', 12), ('a', 22), ('b', 12), ('c', 8)])

print(join(a, b).collect())

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>